## Dicom to Image

In [1]:
import cv2
from PIL import Image
import numpy as np
import os
import pandas as pd
import pydicom
import matplotlib.pyplot as plt

* csv : csv에서 Implant-Manufacturer을 Label로 사용

In [2]:
df = pd.read_csv('/workspace/label/20220918_nopwd.csv', encoding='cp949')

In [3]:
df.info

<bound method DataFrame.info of          데이터명 Case_No.          ID  Sex  Age Insertion  date Removal date  \
0     데이터 표기값   0001 ~         NaN  NaN  NaN             NaN          NaN   
1         NaN        1  28170976.0    F   34        2000.6.1    2007.3.13   
2         NaN        1  28170976.0    F   34        2000.6.1    2007.3.13   
3         NaN        1  28170976.0    F   34        2000.6.1    2007.3.13   
4         NaN        2  24915920.0    F   50        2007.8.8          NaN   
...       ...      ...         ...  ...  ...             ...          ...   
1973      NaN      698  49657090.0    F  36세        2018.2.6          NaN   
1974      NaN      699  49830059.0    F  35세        2018.2.6          NaN   
1975      NaN      699  49830059.0    F  35세        2018.2.6          NaN   
1976      NaN      700  24731494.0    F  42세        2018.2.8          NaN   
1977      NaN      700  24731494.0    F  42세        2018.2.8          NaN   

                                           

In [4]:
len(set(df['ID']))

701

In [5]:
len(df)

1978

→ 701명 1978장의 데이터

In [6]:
df.head()

,데이터명,Case_No.,ID,Sex,Age,Insertion date,Removal date,Implant,Unnamed: 8,Unnamed: 9,H/W,sono date,Period,영상갯수,Exclusion여부
0,데이터 표기값,0001 ~,NaN,NaN,NaN,NaN,NaN,Manufacturer\n(Mentor: 0 \nBellaGel: 1 \nAller...,Texture\n(Smooth: 0\nMicrotextured: 1\nTexture...,Rupture\n(Intact: 0\nRuptured: 1),Sonon: 0\nCanon: 1\nGE:2\nAccuvix:3\nHitachi:4...,NaN,(단위: 년),NaN,Inclusion(=0)\nExclusion(=1)
1,NaN,1,28170976.0,F,34,2000.6.1,2007.3.13,3,0,0,2,1.0,1,4.0,0
2,NaN,1,28170976.0,F,34,2000.6.1,2007.3.13,3,0,0,2,3.0,3,1.0,0
3,NaN,1,28170976.0,F,34,2000.6.1,2007.3.13,3,0,1,2,7.0,7,5.0,0
4,NaN,2,24915920.0,F,50,2007.8.8,NaN,0,0,0,3,1.0,1,6.0,0


* Exclusion = 1 제외

In [7]:
df['Exclusion여부'] = df['Exclusion여부'].astype('str')

In [8]:
df['Exclusion여부'].unique()

array(['Inclusion(=0)\nExclusion(=1)', '0', '1', 'nan', '7'], dtype=object)

In [9]:
(df['Exclusion여부'] == '0')

0       False
1        True
2        True
3        True
4        True
        ...  
1973    False
1974    False
1975    False
1976    False
1977    False
Name: Exclusion여부, Length: 1978, dtype: bool

In [10]:
(df['Exclusion여부'] == 'nan')

0       False
1       False
2       False
3       False
4       False
        ...  
1973     True
1974    False
1975     True
1976     True
1977     True
Name: Exclusion여부, Length: 1978, dtype: bool

In [11]:
df = df[(df['Exclusion여부'] == '0') | (df['Exclusion여부'] == 'nan')] ; df.head()

,데이터명,Case_No.,ID,Sex,Age,Insertion date,Removal date,Implant,Unnamed: 8,Unnamed: 9,H/W,sono date,Period,영상갯수,Exclusion여부
1,NaN,1,28170976.0,F,34,2000.6.1,2007.3.13,3,0,0,2,1.0,1,4.0,0
2,NaN,1,28170976.0,F,34,2000.6.1,2007.3.13,3,0,0,2,3.0,3,1.0,0
3,NaN,1,28170976.0,F,34,2000.6.1,2007.3.13,3,0,1,2,7.0,7,5.0,0
4,NaN,2,24915920.0,F,50,2007.8.8,NaN,0,0,0,3,1.0,1,6.0,0
5,NaN,2,24915920.0,F,50,2007.8.8,NaN,0,0,0,4,2.0,2,2.0,0


In [12]:
len(df)

1920

In [13]:
len(df['ID'].unique())

696

→ 701명, 1978개 중 5명, 58개 제외

* 사용할 데이터만 남기기

In [14]:
data = df[1:][['ID', 'Implant']] ; data

,ID,Implant
2,28170976.0,3
3,28170976.0,3
4,24915920.0,0
5,24915920.0,0
6,24915920.0,0
...,...,...
1972,45095830.0,1
1973,49657090.0,0
1975,49830059.0,0
1976,24731494.0,2


* 중복 제거 : 중복 없음

In [15]:
data = data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [16]:
len(set(data['ID']))

696

* 데이터 환자수 확인

In [17]:
dicom_path = '/workspace/dataset_new'

In [18]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [19]:
get_files_count(dicom_path)

700

* image save : dicom 이미지를 array → Image로 변환하여 저장<br>
이때 image의 환자명을 csv와 매칭하여 Implant 1번이 Label이 되게 한다.

In [20]:
img_path = '/workspace/dataset_new'

* dcm to array : dataframe으로 만들어서 train_test_split 후 저장

In [21]:
dcm_to_arr =  pd.DataFrame()

In [22]:
t_ct = []
# 이미지로 변환 성공한 url
f_ct = []
# 이미지로 변환 실패한 url
dcm_all = []
# dicom 전체 url

## True df 구성 ##
img_arr = []
# img array
lb_arr = []
# label array
path_arr = []
# path array

## False df 구성 ##
#f_img_arr = []
# img array
f_lb_arr = []
# label array
#f_path_arr = []
# path array
for top, _, f in os.walk(img_path):
    # img_path에 접근하여 폴더별로 이미지에 접근한다.
    # top은 img_path/세부경로
    # f는 환자별 image file 이름
    for ix, filename in enumerate(f):
        # 그중 f는 각 환자 폴더 안의 이미지 전부의 주소이다.
        # filename은 그 중 하나에 접근.
        url = os.path.join(top,filename)
        # 세부 이미지 경로
        id = int(url.split('/')[3])
        # 환자 number
        dcm_all.append(url)
        break

In [23]:
dcm_all

['/workspace/dataset_new/52017201/22-03-21_1100003 (2).dcm',
 '/workspace/dataset_new/50676642/21-12-14_1200003 (3).dcm',
 '/workspace/dataset_new/32430451/22-03-17_1000003 (3).dcm',
 '/workspace/dataset_new/51870896/22-02-080005.dcm',
 '/workspace/dataset_new/52957284/22-07-27_0000003 (2).dcm',
 '/workspace/dataset_new/53203858/21-01-140004.dcm',
 '/workspace/dataset_new/31278775/211109_1100007.dcm',
 '/workspace/dataset_new/41947249/21-11-24_0000003 (2).dcm',
 '/workspace/dataset_new/52698686/21-08-20_0000006 (5).dcm',
 '/workspace/dataset_new/51162780/21-06-22_2200001.dcm',
 '/workspace/dataset_new/54979064/2208020006.dcm',
 '/workspace/dataset_new/45095830/2111050001.dcm',
 '/workspace/dataset_new/51869043/2103170001.dcm',
 '/workspace/dataset_new/29186431/21-10-06_000.dcm',
 '/workspace/dataset_new/52639391/22-03-18_0000003 (2).dcm',
 '/workspace/dataset_new/52228645/22-03-04_000.dcm',
 '/workspace/dataset_new/42262770/2112090001.dcm',
 '/workspace/dataset_new/35940010/2202280002.

In [24]:
dcm = pydicom.dcmread('/workspace/dataset_new/13716279/21-11-08_0200002 (2).dcm')

In [25]:
img = dcm.pixel_array

In [26]:
t_ct = []
# 이미지로 변환 성공한 url
f_ct = []
# 이미지로 변환 실패한 url
dcm_all = []
# dicom 전체 url

## True df 구성 ##
img_arr = []
# img array
lb_arr = []
# label array
path_arr = []
# path array

## False df 구성 ##
#f_img_arr = []
# img array
f_lb_arr = []
# label array
#f_path_arr = []
# path array



for top, _, f in os.walk(img_path):
    # img_path에 접근하여 폴더별로 이미지에 접근한다.
    # top은 img_path/세부경로
    # f는 환자별 image file 이름
    for ix, filename in enumerate(f):
        # 그중 f는 각 환자 폴더 안의 이미지 전부의 주소이다.
        # filename은 그 중 하나에 접근.
        url = os.path.join(top,filename)
        # 세부 이미지 경로
        id = int(url.split('/')[3])
        # 환자 number
        dcm_all.append(url)
        # dicom 전체 url
        try:
            label = data[data['ID'] == id].iloc[0].Implant
            # label을 csv의 Implant에 해당하는 값으로 지정
            dcm = pydicom.dcmread(url)
            # dcm read
            img = dcm.pixel_array
            # try 적용 영역 : 변환이 안되면 except 문으로
        
        
            lb_arr.append(label)
            # label 순서대로 더하기 : df에 사용
            img_arr.append(img)
            # array 순서대로 더하기 : df에 사용
            # dcm2array
            save_img = Image.fromarray(img)
            # array2img
            t_ct.append(url)
            # 성공적으로 읽어온 rul
            save_path = '/workspace/pre_dataset/' + label
            # 이미지로 저장할 label url : label별로 저장
            img_save_path = save_path + '/' + url.split('/')[-2]+ str(ix) + '.jpg'
            path_arr.append(img_save_path)
            # 이미지 세부 경로 순서대로 더하기 : df에 사용
            # label url에 세부적으로 저장할 img url, 중복된 이미지명 존재로 인해 index도 이름에 같이 넣어준다.
            #if not os.path.isdir(save_path):
             #   os.makedirs(save_path)
                # 폴더가 없으면 생성
           # save_img.save(img_save_path)
            # 이미지 저장

        except:
            f_ct.append(url)
            try:
                f_lb_arr.append(label)
            except:
                f_lb_arr.append('NaN')
            # dicom으로 읽지 못한 url 저장

            print('error: ' , url)

error:  /workspace/dataset_new/29473483/8-1104060002.jpg
error:  /workspace/dataset_new/29473483/8-1104060001.jpg
error:  /workspace/dataset_new/29473483/8-0604130000.jpg
error:  /workspace/dataset_new/29473483/8-0904070000.jpg
error:  /workspace/dataset_new/29473483/8-1104060000.jpg
error:  /workspace/dataset_new/29473483/8-0604130001.jpg
error:  /workspace/dataset_new/29473483/8-1004060000.jpg
error:  /workspace/dataset_new/29473483/8-0604130002.jpg
error:  /workspace/dataset_new/29473483/9-0404010001.jpg
error:  /workspace/dataset_new/29473483/8-1004060002.jpg
error:  /workspace/dataset_new/29473483/8-1004060001.jpg
error:  /workspace/dataset_new/29473483/9-0404010000.jpg
error:  /workspace/dataset_new/29473483/8-0704190000.jpg
error:  /workspace/dataset_new/48368645/16-2204150000.jpg
error:  /workspace/dataset_new/48368645/16-2104150000.jpg
error:  /workspace/dataset_new/48368645/16-2204150001.jpg
error:  /workspace/dataset_new/48368645/16-2110250000.jpg
error:  /workspace/dataset_

In [47]:
t_ct = []
# 이미지로 변환 성공한 url
f_ct = []
# 이미지로 변환 실패한 url
dcm_all = []
# dicom 전체 url

## True df 구성 ##
img_arr = []
# img array
lb_arr = []
# label array
path_arr = []
# path array
t_id = []

## False df 구성 ##
f_lb_arr = []
# label array



for top, _, f in os.walk(img_path):
    # img_path에 접근하여 폴더별로 이미지에 접근한다.
    # top은 img_path/세부경로
    # f는 환자별 image file 이름
    for filename in f:
        # 그중 f는 각 환자 폴더 안의 이미지 전부의 주소이다.
        # filename은 그 중 하나에 접근.
        url = os.path.join(top,filename)
        # 세부 이미지 경로
        id = int(url.split('/')[3])
        # 환자 number
        dcm_all.append(url)
        # dicom 전체 url
        try:
            label = data[data['ID'] == id].iloc[0].Implant
            # label을 csv의 Implant에 해당하는 값으로 지정
            dcm = pydicom.dcmread(url)
            # dcm read
            img = dcm.pixel_array
            # try 적용 영역 : 변환이 안되면 except 문으로
            lb_arr.append(label)
            # label 순서대로 더하기 : df에 사용
            t_ct.append(url)
            # 성공적으로 읽어온 rul
            t_id.append(id)
            save_path = url + '/' + str(label)
            path_arr.append(img_save_path)

        except:
            f_ct.append(url)
            try:
                check = url.replace('.', '/').split('/')[-1]
                img = dcm.pixel_array
                f_lb_arr.append(check)
            except:
                f_lb_arr.append(check)
            print('error: ' , url)

error:  /workspace/dataset_new/29473483/8-1104060002.jpg
error:  /workspace/dataset_new/29473483/8-1104060001.jpg
error:  /workspace/dataset_new/29473483/8-0604130000.jpg
error:  /workspace/dataset_new/29473483/8-0904070000.jpg
error:  /workspace/dataset_new/29473483/8-1104060000.jpg
error:  /workspace/dataset_new/29473483/8-0604130001.jpg
error:  /workspace/dataset_new/29473483/8-1004060000.jpg
error:  /workspace/dataset_new/29473483/8-0604130002.jpg
error:  /workspace/dataset_new/29473483/9-0404010001.jpg
error:  /workspace/dataset_new/29473483/8-1004060002.jpg
error:  /workspace/dataset_new/29473483/8-1004060001.jpg
error:  /workspace/dataset_new/29473483/9-0404010000.jpg
error:  /workspace/dataset_new/29473483/8-0704190000.jpg
error:  /workspace/dataset_new/48368645/16-2204150000.jpg
error:  /workspace/dataset_new/48368645/16-2104150000.jpg
error:  /workspace/dataset_new/48368645/16-2204150001.jpg
error:  /workspace/dataset_new/48368645/16-2110250000.jpg
error:  /workspace/dataset_

* 읽힌 파일과 읽지 못한 파일

In [48]:
len(dcm_all)

4947

In [49]:
len(t_ct)

4760

In [50]:
len(f_ct)

187

In [97]:
187-18

169

* jpg check

In [51]:
ecp = []
for pt in f_ct:
    if pt.split('.')[-1] == 'jpg':
        pass
    else:
        ecp.append(pt)

In [52]:
ecp

['/workspace/dataset_new/49479663/2202250002.dcm',
 '/workspace/dataset_new/49479663/2202250003.dcm',
 '/workspace/dataset_new/49479663/2202250001.dcm',
 '/workspace/dataset_new/49479663/200804.dcm',
 '/workspace/dataset_new/27578096/20-05-07_100.dcm',
 '/workspace/dataset_new/27578096/20-05-07_110.dcm',
 '/workspace/dataset_new/54721119/2203080001.dcm',
 '/workspace/dataset_new/54721119/2203080002.dcm',
 '/workspace/dataset_new/49019061/2111180002.dcm',
 '/workspace/dataset_new/49019061/2111180003.dcm',
 '/workspace/dataset_new/49019061/2111180001.dcm',
 '/workspace/dataset_new/43654170/21-07-030007.dcm',
 '/workspace/dataset_new/43654170/21-07-030001.dcm',
 '/workspace/dataset_new/43654170/21-07-030005.dcm',
 '/workspace/dataset_new/43654170/21-07-030002.dcm',
 '/workspace/dataset_new/43654170/21-07-030004.dcm',
 '/workspace/dataset_new/43654170/21-07-030003.dcm',
 '/workspace/dataset_new/43654170/21-07-030006.dcm']

In [53]:
ct_f_num = []
for i in f_ct:
    ct_f_num.append(i.split('/')[3])

In [54]:
set(ct_f_num)

{'24915920',
 '27578096',
 '27804531',
 '28170976',
 '29473483',
 '30035829',
 '37782140',
 '39114039',
 '40663029',
 '42668455',
 '43654170',
 '44065672',
 '44289834',
 '45554175',
 '47772931',
 '48368645',
 '49019061',
 '49479663',
 '49513323',
 '50032204',
 '50400193',
 '51039936',
 '51603856',
 '53007973',
 '54025873',
 '54721119'}

In [55]:
len(set(ct_f_num))

26

* csv와 매칭

In [56]:
id1 = 54721119

In [57]:
id1 in data['ID']

False

In [58]:
id2 = 49019061

In [59]:
id2 in data['ID']

False

In [60]:
id3 = 43654170

In [61]:
id3 in data['ID']

False

In [62]:
id4 = 27578096

In [63]:
id4 in data['ID']

False

In [64]:
id5 = 49479663

In [65]:
id5 in data['ID']

False

* csv에 존재하지않는 환자 number 21명 그리고 나머지 파일은 5명은 jpg로 존재한다.<br>
→ 4760장으로 진행

* data frame 구성

In [66]:
df = pd.DataFrame([ x for x in zip(t_ct,lb_arr, t_id)], columns = ['url', 'label', 'id'])

In [67]:
df.head()

,url,label,id
0,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
1,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
2,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
3,/workspace/dataset_new/52017201/21-01-15_11000...,1,52017201
4,/workspace/dataset_new/52017201/21-01-15_11000...,1,52017201


In [208]:
df.to_csv('df.csv', index=False)

In [75]:
len(df['id'].unique())

675

* 읽지 못한 url

In [68]:
f_ct

['/workspace/dataset_new/29473483/8-1104060002.jpg',
 '/workspace/dataset_new/29473483/8-1104060001.jpg',
 '/workspace/dataset_new/29473483/8-0604130000.jpg',
 '/workspace/dataset_new/29473483/8-0904070000.jpg',
 '/workspace/dataset_new/29473483/8-1104060000.jpg',
 '/workspace/dataset_new/29473483/8-0604130001.jpg',
 '/workspace/dataset_new/29473483/8-1004060000.jpg',
 '/workspace/dataset_new/29473483/8-0604130002.jpg',
 '/workspace/dataset_new/29473483/9-0404010001.jpg',
 '/workspace/dataset_new/29473483/8-1004060002.jpg',
 '/workspace/dataset_new/29473483/8-1004060001.jpg',
 '/workspace/dataset_new/29473483/9-0404010000.jpg',
 '/workspace/dataset_new/29473483/8-0704190000.jpg',
 '/workspace/dataset_new/48368645/16-2204150000.jpg',
 '/workspace/dataset_new/48368645/16-2104150000.jpg',
 '/workspace/dataset_new/48368645/16-2204150001.jpg',
 '/workspace/dataset_new/48368645/16-2110250000.jpg',
 '/workspace/dataset_new/48368645/16-2204150002.jpg',
 '/workspace/dataset_new/48368645/16-2104

In [69]:
f_df = pd.DataFrame([ x for x in zip(f_ct, f_lb_arr)], columns = ['url', 'check'])

In [70]:
f_df

,url,check
0,/workspace/dataset_new/29473483/8-1104060002.jpg,jpg
1,/workspace/dataset_new/29473483/8-1104060001.jpg,jpg
2,/workspace/dataset_new/29473483/8-0604130000.jpg,jpg
3,/workspace/dataset_new/29473483/8-0904070000.jpg,jpg
4,/workspace/dataset_new/29473483/8-1104060000.jpg,jpg
...,...,...
182,/workspace/dataset_new/24915920/2-0904250000.jpg,jpg
183,/workspace/dataset_new/24915920/2-0811030001.jpg,jpg
184,/workspace/dataset_new/24915920/2-0811030003.jpg,jpg
185,/workspace/dataset_new/24915920/2-0904250001.jpg,jpg


In [212]:
f_df.to_csv('f_df.csv', index=False)

* data leakage 방지 split

In [71]:
df.head()

,url,label,id
0,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
1,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
2,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
3,/workspace/dataset_new/52017201/21-01-15_11000...,1,52017201
4,/workspace/dataset_new/52017201/21-01-15_11000...,1,52017201


→ 동일한 id의 파일은 train, valid, test중 모두 하나에 속해야함.

In [34]:
df = pd.read_csv('/workspace/Pipeline/1_data_preprocessing/df.csv')

In [76]:
df

,url,label,id
0,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
1,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
2,/workspace/dataset_new/52017201/22-03-21_11000...,1,52017201
3,/workspace/dataset_new/52017201/21-01-15_11000...,1,52017201
4,/workspace/dataset_new/52017201/21-01-15_11000...,1,52017201
...,...,...,...
4755,/workspace/dataset_new/34716454/20-07-22_11000...,1,34716454
4756,/workspace/dataset_new/34716454/20-07-22_11000...,1,34716454
4757,/workspace/dataset_new/34716454/22-08-19_11000...,1,34716454
4758,/workspace/dataset_new/34716454/19-12-31_11000...,1,34716454


In [77]:
len(set(list(df['id'])))

675

* train valid split

In [78]:
df['id']

0       52017201
1       52017201
2       52017201
3       52017201
4       52017201
          ...   
4755    34716454
4756    34716454
4757    34716454
4758    34716454
4759    34716454
Name: id, Length: 4760, dtype: int64

In [79]:
df_du = df[['label', 'id']].drop_duplicates('id')

In [80]:
df_du

,label,id
0,1,52017201
6,1,50676642
11,1,32430451
14,0,51870896
19,0,52957284
...,...,...
4733,0,51965941
4736,0,52757439
4739,0,53604398
4740,1,50205451


In [81]:
from sklearn.model_selection import train_test_split

In [82]:
x_train, x_valid ,y_train, y_valid = train_test_split(df_du['id'], df_du['label'], test_size=0.3, shuffle=True, stratify=df_du['label'], random_state=2022)

In [83]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

(472,) (472,) (203,) (203,)


* valid test split

In [84]:
x_valid, x_test ,y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.5, shuffle=True, stratify=y_valid, random_state=2022)

In [85]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(472,) (472,) (101,) (101,) (102,) (102,)


In [238]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(472,) (472,) (101,) (101,) (102,) (102,)


In [88]:
472+101+102

675

* train save

In [86]:
df_x_train = df[df['id'].map(lambda x : x in list(x_train))]

In [87]:
len(df_x_train)

3281

In [271]:
for idx, (label, path) in enumerate(zip(df_x_train['label'], df_x_train['url'])):
    dcm = pydicom.dcmread(path)
    img = dcm.pixel_array
    save_img = Image.fromarray(img)
    id = path.replace('.', '/').split('/')[-3]
    fd_path = '/workspace/pre_dataset/train/' + str(label)
    img_path = fd_path + '/' + str(id) + '-' + str(idx) +'.jpg'
    if not os.path.isdir(fd_path):
                os.makedirs(fd_path)
    save_img.save(img_path)

* valid save

In [272]:
df_x_valid = df[df['id'].map(lambda x : x in list(x_valid))]

In [273]:
for idx, (label, path) in enumerate(zip(df_x_valid['label'], df_x_valid['url'])):
    dcm = pydicom.dcmread(path)
    img = dcm.pixel_array
    save_img = Image.fromarray(img)
    id = path.replace('.', '/').split('/')[-3]
    fd_path = '/workspace/pre_dataset/valid/' + str(label)
    img_path = fd_path + '/' + str(id) + '-' + str(idx) +'.jpg'
    if not os.path.isdir(fd_path):
                os.makedirs(fd_path)
    save_img.save(img_path)

* test save

In [274]:
df_x_test = df[df['id'].map(lambda x : x in list(x_test))]

In [275]:
for idx, (label, path) in enumerate(zip(df_x_test['label'], df_x_test['url'])):
    dcm = pydicom.dcmread(path)
    img = dcm.pixel_array
    save_img = Image.fromarray(img)
    id = path.replace('.', '/').split('/')[-3]
    fd_path = '/workspace/pre_dataset/test/' + str(label)
    img_path = fd_path + '/' + str(id) + '-' + str(idx) +'.jpg'
    if not os.path.isdir(fd_path):
                os.makedirs(fd_path)
    save_img.save(img_path)

* 갯수 확인

In [42]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [43]:
train_0 = get_files_count('/workspace/pre_dataset/train/0')
train_1 = get_files_count('/workspace/pre_dataset/train/1')
train_2 = get_files_count('/workspace/pre_dataset/train/2')
train_3 = get_files_count('/workspace/pre_dataset/train/3')

In [44]:
print(train_0, train_1, train_2, train_3)

0 0 0 0


In [284]:
1808 + 1175 + 229 + 37

3249

In [281]:
valid_0 = get_files_count('/workspace/pre_dataset/valid/0')
valid_1 = get_files_count('/workspace/pre_dataset/valid/1')
valid_2 = get_files_count('/workspace/pre_dataset/valid/2')
valid_3 = get_files_count('/workspace/pre_dataset/valid/3')

In [282]:
print(valid_0, valid_1, valid_2, valid_3)

409 309 49 11


In [283]:
409 + 309 + 49 + 11

778

In [285]:
test_0 = get_files_count('/workspace/pre_dataset/test/0')
test_1 = get_files_count('/workspace/pre_dataset/test/1')
test_2 = get_files_count('/workspace/pre_dataset/test/2')
test_3 = get_files_count('/workspace/pre_dataset/test/3')

In [286]:
print(test_0, test_1, test_2, test_3)

387 283 57 6


In [287]:
387 + 283 + 57 + 6

733

In [288]:
len(df)

4760

In [289]:
3249 + 778 + 733

4760

→ 갯수 이슈 없음